# Objetivo

Dada uma planilha em Excel, inserir os dados da planilha num banco de dados PostgreSQL.

Dica: você pode usar Python junto com psycopg2 para se conectar no PostgreSQL.
    
Passos:

* Criar uma virtualenv

* Instalar psycopg2

* Ler os dados da planilha Excel

* Tratar os dados, se necessário
* Inserir os dados no banco PostgreSQL
* Coloque seu projeto no Gitlab, ou Github pessoal e me manda o link do repositório

# Dependências 

## Biblioteca

In [1]:
import psycopg2
import pandas as pd
from decouple import config

## Dados e Constantes

In [2]:
df = pd.read_excel('categorias.xlsx')
df_1 = pd.read_excel('produtos.xlsx')
df_2 = pd.read_excel('produtos-com-categoria.xlsx')

In [3]:
#tabela categoria
df.head()

,id,categoria
0,101,bebidas
1,210,sobremesas
2,314,cereais
3,420,frios
4,598,limpeza


In [4]:
#Tabela produtos
df_1.head()

,id,produto,preco
0,1,uva,2.59
1,2,Suco de uva,10.47
2,3,Suco de manga,10.47
3,4,Suco de maça,10.47
4,5,Suco de laranja,10.47


In [5]:
#Produtos com categorias
df_2.head()

,id,produto,preco,categoria
0,1,uva,2.59,670
1,2,Suco de uva,10.47,101
2,3,Suco de manga,10.47,101
3,4,Suco de maça,10.47,101
4,5,Suco de laranja,10.47,101


In [6]:
df_2.columns

Index(['id', 'produto', 'preco', 'categoria'], dtype='object')

## Inserindo os dados no banco Psql

In [7]:
try:
    connection = psycopg2.connect(user = "my_usuario",
                                  password = config("PASSWORD"),
                                  host = "127.0.0.1",
                                  port = "5432",
                                  database = "my_db")

    cursor = connection.cursor()
    # Print PostgreSQL Connection properties
    print ( connection.get_dsn_parameters(),"\n")

    # Print PostgreSQL version
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record,"\n")

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)

{'user': 'my_usuario', 'dbname': 'my_db', 'host': '127.0.0.1', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.7 (Ubuntu 11.7-0ubuntu0.19.10.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.2.1-9ubuntu2) 9.2.1 20191008, 64-bit',) 



In [8]:
cur = connection.cursor()

## Tabela `categoria`

In [9]:
#Criando tabela para categoria
cur.execute("""CREATE TABLE IF NOT EXISTS categoria (id SERIAL PRIMARY KEY, nome VARCHAR(50));""")

In [10]:
for i,row in df.iterrows():
    name = row["categoria"]
    cur.execute(f"INSERT INTO categoria (nome) VALUES ('{name}')")

In [11]:
connection.commit()

In [12]:
cur.execute("SELECT * FROM categoria;")
cur.fetchall()

[(1, 'bebidas'),
 (2, 'sobremesas'),
 (3, 'cereais'),
 (4, 'frios'),
 (5, 'limpeza'),
 (6, 'hortifruti'),
 (7, 'perfumaria'),
 (8, 'mercearia'),
 (9, 'bazar'),
 (10, 'pet shop')]

## Tabela `produtos`

In [13]:
#Criando tabela para produtos. 
cur.execute("""CREATE TABLE IF NOT EXISTS produtos(id SERIAL PRIMARY KEY, nome_produto VARCHAR(50), valor_produtos decimal);""")

In [14]:
for i, row in df_1.iterrows():
    nome_produto = row["produto"]
    valor_produtos = row["preco"]
    cur.execute(f"INSERT INTO produtos (nome_produto, valor_produtos) VALUES ('{nome_produto}','{valor_produtos}')")

In [15]:
connection.commit()

In [16]:
cur.execute("SELECT * FROM produtos;")
cur.fetchall()

[(1, 'uva', Decimal('2.59')),
 (2, 'Suco de uva', Decimal('10.47')),
 (3, 'Suco de manga', Decimal('10.47')),
 (4, 'Suco de maça', Decimal('10.47')),
 (5, 'Suco de laranja', Decimal('10.47')),
 (6, 'shampoo', Decimal('3.97')),
 (7, 'sardinha enlatada', Decimal('8.63')),
 (8, 'sandália Havaianas', Decimal('4.56')),
 (9, 'sabonete', Decimal('2.75')),
 (10, 'Sabão em pó', Decimal('17.93')),
 (11, 'Sabão em barra', Decimal('1.75')),
 (12, 'Ração para gato', Decimal('14.96')),
 (13, 'Ração para cachorro', Decimal('8.63')),
 (14, 'quinoa', Decimal('7.12')),
 (15, 'Queijo mussarela', Decimal('5.33')),
 (16, 'Queijo frescal', Decimal('5.55')),
 (17, 'Pudim', Decimal('2.55')),
 (18, 'Petit Gateau', Decimal('6.77')),
 (19, 'óleo de soja', Decimal('4.6')),
 (20, 'óleo de girassol', Decimal('16.85')),
 (21, 'mortadela', Decimal('6.39')),
 (22, 'melão', Decimal('1.12')),
 (23, 'melancia', Decimal('2.1')),
 (24, 'manga', Decimal('15.41')),
 (25, 'maionese', Decimal('3.32')),
 (26, 'macarrão', Decima

## Tabela `produtos_categoria`

In [17]:
#Produtos com categoria
cur.execute("""CREATE TABLE IF NOT EXISTS produtos_categoria (id SERIAL PRIMARY KEY, nome_produto VARCHAR(50), valor_produto decimal, categoria_tipo INT REFERENCES categoria(id));""")

In [18]:
connection.commit()

In [ ]:
for i, row in df_2.iterrows():
    nome_produto = row["produto"]
    valor_produto = row["preco"]
    categoria_tipo = row["categoria"]
    # cur.execute(f"INSERT INTO produtos_categoria (nome_produto, valor_produto, categoria_tipo) VALUES ('{nome_produto}','{valor_produto}',{categoria_tipo})")

In [ ]:
# connection.commit()

In [ ]:
# cur.execute("SELECT * FROM produtos_categoria;")
# cur.fetchall()

In [ ]:
#cur.execute("DELETE FROM produtos where id > 64;")
#cur.execute("DROP TABLE IF EXISTS produtos;")
#cur.execute("ALTER TABLE produtos DROP COLUMN valor_produto;")
#cur.execute("ALTER TABLE produtos ADD valor_produtos decimal;")